In [ ]:
#please run in google colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#reading the text file
with open('drive/My Drive/December2019.txt', 'r') as f: #change the file name and path
    stuff = f.read()

In [ ]:
#making the dictionaries from the text file
import ast
a = stuff.index("=")
b = stuff.index("=", a+1)
taxi_string = stuff[a+2:b-17].strip()
taxi_count = ast.literal_eval(taxi_string)
mean_string = stuff[b+2:].strip()
clusters_to_mean = ast.literal_eval(mean_string)

In [ ]:
t = np.array(sorted(list(taxi_count.keys())))
num_of_areas = 50

import numpy as np
from scipy.optimize import least_squares

#defining functions for nonlinear regression

def fun(x, t, y):
    return x[0] * np.exp(-x[1] * t) * np.sin(x[2] * t) - y

def generate_data(t, A, sigma, omega, noise=0, n_outliers=0, random_state=0):
    y = A * np.exp(-sigma * t) * np.sin(omega * t)
    rnd = np.random.RandomState(random_state)
    error = noise * rnd.randn(t.size)
    outliers = rnd.randint(0, t.size, n_outliers)
    error[outliers] *= 35
    return y + error

x0 = np.ones(3)

predict_taxi_counts = {}

#doing the nonlinear regression on each cluster over the time intervals

for x in range(num_of_areas):
  y = []
  for i in range(len(t)):
    if x in taxi_count[t[i]].keys():
      y.append(taxi_count[t[i]][x])
    else:
      y.append(0)
  y = np.array(y)
  res_robust = least_squares(fun, x0, loss='soft_l1', f_scale=0.1, args=(t, y))
  y_robust = generate_data(t, *res_robust.x)
  for i in range(len(y_robust)):
    if y_robust[i] < 0.5: num = 0
    else: num = int(y_robust)
    if t[i] in predict_taxi_counts.keys():
      temp = predict_taxi_counts[t[i]]
      temp[x] = num
      predict_taxi_counts[t[i]] = temp
    else:
      predict_taxi_counts[t[i]] = {x: num}

In [ ]:
with open('/content/drive/My Drive/December2020.txt', 'w') as writefile: #change file name and path
    writefile.write("taxi_count = ")
    writefile.write(str(predict_taxi_counts))
    writefile.write("clusters_to_mean = ")
    writefile.write(str(clusters_to_mean))